In [28]:
%load_ext autoreload
%autoreload 2

from utils import *
pdf_path = "mcs2024.pdf"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Extract the material lists

In [ ]:
df = extract_word_positions(pdf_path, 0)
complexe_df = complexe_word(df, 2)

complexe_df.sample(2)

In [ ]:
result_df = extract_cols(complexe_df, 'x0', 2)

In [ ]:
result_df

In [ ]:
result_df['max_top'].min()

In [ ]:
import pdfplumber

with pdfplumber.open("mcs2024.pdf") as pdf:
    im = pdf.pages[0].to_image(resolution=250)

im.draw_line([(575, result_df['max_top'].min()), (44, result_df['max_top'].min())], )
im.draw_line([(575, result_df['min_bottom'].max()), (44, result_df['min_bottom'].max())], )

In [ ]:
materials_list = complexe_df[(complexe_df['top'] > int(result_df['max_top'].min())) & (complexe_df['bottom'] < 630)]['text'].tolist()

len(materials_list) #🟢

In [ ]:
materials_list

In [ ]:
with open('json_files\materials_list.json', 'w') as json_file:
    json.dump(materials_list, json_file)

In [ ]:
with open('json_files/materials_list.json', 'r') as json_file:
    materials_list = json.load(json_file)

In [ ]:
materials_list #🟢

## Match each material with its pages

In [ ]:
# Ouvrir le PDF
elements = materials_list
l = []

with pdfplumber.open(pdf_path) as pdf:
    # Dictionnaire pour stocker les pages correspondant à chaque élément
    pdf_sections = {material: {'title': pd.DataFrame(), 'pages': []} for material in materials_list}
    
    # Parcourir chaque page du PDF
    for i in range(33, 209):

        # page = pdf.pages[i]
        largest_text = extract_largest_text(pdf_path, i)['text'].to_list()
        # l += largest_text
        # print(f'▶️ Largest_text: {largest_text}')

        for text in largest_text:
            matched_element = match_element_in_text(materials_list, text)
            if matched_element:
                pdf_sections[matched_element]['title'] = pd.concat([pdf_sections[matched_element]['title'], extract_largest_text(pdf_path, i)], ignore_index=True)
                pdf_sections[matched_element]['pages'].append(i + 1)

# print(f'▶️ Largest_text len: {len(l)}')

In [ ]:
empty_pages_elements = [material for material, details in pdf_sections.items() if not details['pages']]

excluded_elements = {'Palladium'}
empty_pages_elements = list(set(empty_pages_elements) - excluded_elements)

empty_pages_elements

In [ ]:
one_page_elements = [material for material, details in pdf_sections.items() if len(details['pages']) == 1]

one_page_elements

In [ ]:
three_page_elements = [material for material, details in pdf_sections.items() if len(details['pages']) == 3]

three_page_elements

In [ ]:
four_page_elements = [material for material, details in pdf_sections.items() if len(details['pages']) == 4]

four_page_elements

In [ ]:
pdf_sections['Sand and Gravel']['title']

In [ ]:
# def convert_df_to_dict(d):
#     if isinstance(d, pd.DataFrame):
#         return d.to_dict(orient='records')
#     elif isinstance(d, list):
#         return [convert_df_to_dict(i) for i in d]
#     elif isinstance(d, dict):
#         return {k: convert_df_to_dict(v) for k, v in d.items()}
#     else:
#         return d

# serializable_pdf_sections = convert_df_to_dict(pdf_sections)

# # Save the dictionary to a JSON file
# with open('json_files/pdf_sections.json', 'w', encoding='utf-8') as json_file:
#     json.dump(serializable_pdf_sections, json_file, ensure_ascii=False, indent=4)

In [ ]:
# with open('json_files/pdf_sections.json', 'r', encoding='utf-8') as json_file:
#     loaded_pdf_sections = json.load(json_file)

# # Convert serialized dictionaries back to DataFrames
# def convert_dict_to_df(d):
#     if isinstance(d, list) and all(isinstance(i, dict) for i in d):
#         return pd.DataFrame(d)
#     elif isinstance(d, list):
#         return [convert_dict_to_df(i) for i in d]
#     elif isinstance(d, dict):
#         return {k: convert_dict_to_df(v) for k, v in d.items()}
#     else:
#         return d

# pdf_sections = convert_dict_to_df(loaded_pdf_sections)

In [ ]:
pdf_sections   #🟠 sauf for Zirconium!!

## Dictionary: 
{material_name: {
'material_title': df(title, bounding box),
'pages_num': [page1, page2],
'pages_content': [dataframe1, dataframe2],
  'remarks': [remark_df],
  'tables': [[tables_1(table + bounding_box)], [tables_2(table + bounding_box)]]
  }
}

In [ ]:
pdf_sections.keys()

In [ ]:
type(pdf_sections['Abrasives']['title'])

In [ ]:
scraping_base = extract_positions_for_elements(pdf_path, pdf_sections)

In [ ]:
scraping_base.keys()

In [ ]:
scraping_base['Abrasives'].keys()

In [ ]:
scraping_base['Abrasives']['remarks']

In [ ]:
# def convert_df_to_dict(d):
#     if isinstance(d, pd.DataFrame):
#         return d.to_dict(orient='records')
#     elif isinstance(d, list):
#         return [convert_df_to_dict(i) for i in d]
#     elif isinstance(d, dict):
#         return {k: convert_df_to_dict(v) for k, v in d.items()}
#     else:
#         return d

# serializable_scraping_base = convert_df_to_dict(scraping_base)

# # Save the dictionary to a JSON file
# with open('json_files/scraping_base.json', 'w', encoding='utf-8') as json_file:
#     json.dump(serializable_scraping_base, json_file, ensure_ascii=False, indent=4)

In [ ]:
# with open('json_files/scraping_base.json', 'r', encoding='utf-8') as json_file:
#     loaded_scraping_base = json.load(json_file)

# # Convert serialized dictionaries back to DataFrames
# def convert_dict_to_df(d):
#     if isinstance(d, list) and all(isinstance(i, dict) for i in d):
#         return pd.DataFrame(d)
#     elif isinstance(d, list):
#         return [convert_dict_to_df(i) for i in d]
#     elif isinstance(d, dict):
#         return {k: convert_dict_to_df(v) for k, v in d.items()}
#     else:
#         return d

# scraping_base = convert_dict_to_df(loaded_scraping_base)

In [ ]:
scraping_base['Abrasives']  #🟢

## Remark extraction

In [ ]:
scraping_base_bricolage = {k: v for k, v in list(scraping_base.items()) if k != 'Palladium' and k != 'Zirconium'}

In [ ]:
scraping_base_bricolage.keys()

In [ ]:
scraping_base_bricolage['Gallium']['remarks']

In [ ]:
# for material in list(scraping_base_bricolage.keys()):
#   scraping_base_bricolage[material]['remarks'] = pd.DataFrame()

In [ ]:
for material, data in scraping_base_bricolage.items():
    # Ensure pages_content is not empty
    if data['pages_content']:
        # print(f" ▶️ {data['pages_content'][0]}")
        remarks = extract_text_between_delimiters(data['pages_content'][0], pdf_path, data['pages_num'][0])
        data['remarks'] = remarks
    else:
        print(f" ▶️ No pages_content for material: {material}")
    
    data['remarks'] = data['remarks'].drop_duplicates(subset=['text'])

In [ ]:
data['remarks'].columns, data['pages_content'][0].columns

In [ ]:
scraping_base_bricolage['Platinum']['remarks']

In [ ]:
# def convert_df_to_dict(d):
#     if isinstance(d, pd.DataFrame):
#         return d.to_dict(orient='records')
#     elif isinstance(d, list):
#         return [convert_df_to_dict(i) for i in d]
#     elif isinstance(d, dict):
#         return {k: convert_df_to_dict(v) for k, v in d.items()}
#     else:
#         return d

# serializable_scraping_base = convert_df_to_dict(scraping_base_bricolage)

# # Save the dictionary to a JSON file
# with open('json_files/scraping_base.json', 'w', encoding='utf-8') as json_file:
#     json.dump(serializable_scraping_base, json_file, ensure_ascii=False, indent=4)

## Table extraction

In [1]:
# START RUN

In [2]:
%load_ext autoreload
%autoreload 2

from utils import *
pdf_path = "mcs2024.pdf"

In [3]:
with open('json_files/scraping_base.json', 'r', encoding='utf-8') as json_file:
    loaded_scraping_base = json.load(json_file)

# Convert serialized dictionaries back to DataFrames
def convert_dict_to_df(d):
    if isinstance(d, list) and all(isinstance(i, dict) for i in d):
        return pd.DataFrame(d)
    elif isinstance(d, list):
        return [convert_dict_to_df(i) for i in d]
    elif isinstance(d, dict):
        return {k: convert_dict_to_df(v) for k, v in d.items()}
    else:
        return d

scraping_base = convert_dict_to_df(loaded_scraping_base)

In [4]:
for material in list(scraping_base.keys()):
  scraping_base[material]['tables'] = []

In [5]:
import pandas as pd
import warnings

# Suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Suppress specific FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning, message="The 'method' keyword in Series.replace is deprecated")
warnings.filterwarnings("ignore", category=FutureWarning, message="Calling float on a single element Series is deprecated")

mt = 3

for material in list(scraping_base.keys()):
    if isinstance(scraping_base[material]['tables'], pd.DataFrame):
        scraping_base[material]['tables'] = scraping_base[material]['tables'].to_dict(orient='records')
    else:
        scraping_base[material]['tables'] = []

    selected_ps, contents = scraping_base[material]['pages_num'], scraping_base[material]['pages_content']
    i = 0

    for selected_p in selected_ps:
        content = contents[i]
        i += 1
        result = extract_table(selected_p - 1, content, mt)
        if result is None or len(result[0]) == 0:
            continue

        list_of_table_df, list_of_bbox, page = result
        table_dfs = table_to_df(list_of_table_df, list_of_bbox, page, pdf_path)
        if not table_dfs:
            continue

        for table_df, bbox in zip(table_dfs, list_of_bbox):
            table = table_df.sort_values('bottom')
            scraping_base[material]['tables'].append((table.to_dict(orient='records'), bbox, page))

Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.
Dataframe is either not defined or empty. Scaling not applied.


In [6]:
# material_test = ['Arsenic', 'Bismuth', 'Cadmium']

# material_test = ['Boron', 'Iron Ore', 'Strontium']

# material_test = ['Sand and Gravel', 'Stone']

# material_test = ['Chromium']

In [7]:
# import pandas as pd
# import warnings

# # Suppress SettingWithCopyWarning
# pd.options.mode.chained_assignment = None

# # Suppress specific FutureWarnings
# warnings.filterwarnings("ignore", category=FutureWarning, message="The 'method' keyword in Series.replace is deprecated")
# warnings.filterwarnings("ignore", category=FutureWarning, message="Calling float on a single element Series is deprecated")

# # Your existing code
# for material in material_test:
#     if isinstance(scraping_base[material]['tables'], pd.DataFrame):
#         scraping_base[material]['tables'] = scraping_base[material]['tables'].to_dict(orient='records')
#     else:
#         scraping_base[material]['tables'] = []

#     selected_ps, contents = scraping_base[material]['pages_num'], scraping_base[material]['pages_content']
#     i = 0

#     for selected_p in selected_ps:
#         content = contents[i]
#         i += 1
#         result = extract_table(selected_p - 1, content, 2)
#         if result is None or len(result[0]) == 0:
#             continue

#         list_of_table_df, list_of_bbox, page = result
#         table_dfs, combined_words = table_to_df(list_of_table_df, list_of_bbox, page, pdf_path)
#         if not table_dfs:
#             continue

#         for table_df, bbox in zip(table_dfs, list_of_bbox):
#             table = table_df.sort_values('bottom')
#             # print(table)
#             # print("*"*40)
#             # print()
#             scraping_base[material]['tables'].append((table.to_dict(orient='records'), bbox, page))

In [8]:
result

([            text          x0          x1        top        doctop     bottom  \
  58      Refinery  262.620000  302.693988  115.66752  126078.00108  125.68752   
  59   productione  305.460000  361.320288  114.39648  126076.73004  125.68752   
  61      Reserves  482.340000  526.860000  114.39648  126076.73004  125.68752   
  62          2022  274.200000  296.511120  126.64752  126088.98108  136.66752   
  63          2023  346.200000  368.511120  126.64752  126088.98108  136.66752   
  ..           ...         ...         ...        ...           ...        ...   
  123        total   83.041728  101.947464  291.13650  126253.47006  301.15650   
  124    (rounded)  104.702964  148.125636  291.13650  126253.47006  301.15650   
  125       13,310  267.782880  296.451120  290.04600  126252.37956  301.15650   
  126       13,600  339.782880  368.452260  290.04600  126252.37956  301.15650   
  127       95,000  499.860000  530.512908  291.13650  126253.47006  301.15650   
  
       uprigh

In [9]:
# STOP

In [10]:
# result[0][0]['text'].to_list()
# result

In [11]:
def convert_df_to_dict(d):
    if isinstance(d, pd.DataFrame):
        return d.to_dict(orient='records')
    elif isinstance(d, list):
        return [convert_df_to_dict(i) for i in d]
    elif isinstance(d, dict):
        return {k: convert_df_to_dict(v) for k, v in d.items()}
    else:
        return d

serializable_scraping_base = convert_df_to_dict(scraping_base)

# Save the dictionary to a JSON file
with open('json_files/scraping_base_with_tables.json', 'w', encoding='utf-8') as json_file:
    json.dump(serializable_scraping_base, json_file, ensure_ascii=False, indent=4)

## Dictionary to excel

In [12]:
with open('json_files/scraping_base_with_tables.json', 'r', encoding='utf-8') as json_file:
    loaded_scraping_base = json.load(json_file)

# Convert serialized dictionaries back to DataFrames
def convert_dict_to_df(d):
    if isinstance(d, list) and all(isinstance(i, dict) for i in d):
        return pd.DataFrame(d)
    elif isinstance(d, list):
        return [convert_dict_to_df(i) for i in d]
    elif isinstance(d, dict):
        return {k: convert_dict_to_df(v) for k, v in d.items()}
    else:
        return d

scraping_base = convert_dict_to_df(loaded_scraping_base)

In [13]:
for material in list(scraping_base.keys()):
    for i in range(len(scraping_base[material]['tables'])):
        table, bbox, page = scraping_base[material]['tables'][i]
        table = table.sort_values('bottom')
        table = table.drop('bottom', axis=1)
        scraping_base[material]['tables'][i][0] = table 

In [14]:
with pd.ExcelWriter('excel_output/all_extracted_tables.xlsx', engine='openpyxl') as writer:
    for material, data in scraping_base.items():

        # Write remarks and tables in the same sheet, but separated by columns
        combined_sheet_name = f"{material}_raw_info"
        
        # Write remarks
        remarks_df = data['remarks'][['text']]
        remarks_df.columns = ['remark']
        remarks_df.to_excel(writer, sheet_name=combined_sheet_name, startrow=0, startcol=0, index=False)

        # Write each table starting from a new column
        start_col = len(remarks_df.columns) + 2  # Adding 2 for separation
        for i, (table_df, bbox, page) in enumerate(data['tables']):
            table_df.to_excel(writer, sheet_name=combined_sheet_name, startrow=0, startcol=start_col, index=False)
            # Increment start_col by the number of columns in the table + 2 for separation
            start_col += len(table_df.columns) + 2

print("Excel file created successfully.")

Excel file created successfully.


In [15]:
remarks_df

,remark
0,"(Data in metric tons, selenium content, unless..."


## PDF visualisation V1

In [16]:
scraping_base.keys()

dict_keys(['Abrasives', 'Fluorspar', 'Mercury', 'Silicon', 'Aluminum', 'Gallium', 'Mica', 'Silver', 'Antimony', 'Garnet', 'Molybdenum', 'Soda Ash', 'Arsenic', 'Gemstones', 'Nickel', 'Stone', 'Asbestos', 'Germanium', 'Niobium', 'Strontium', 'Barite', 'Gold', 'Nitrogen', 'Sulfur', 'Bauxite', 'Graphite', 'Talc', 'Beryllium', 'Gypsum', 'Peat', 'Tantalum', 'Bismuth', 'Hafnium', 'Perlite', 'Tellurium', 'Boron', 'Helium', 'Phosphate Rock', 'Thallium', 'Bromine', 'Indium', 'Platinum', 'Thorium', 'Cadmium', 'Iodine', 'Potash', 'Tin', 'Cement', 'Iron and Steel', 'Pumice', 'Titanium', 'Cesium', 'Iron Ore', 'Quartz', 'Tungsten', 'Chromium', 'Iron Oxide Pigments', 'Rare Earths', 'Vanadium', 'Clays', 'Kyanite', 'Rhenium', 'Vermiculite', 'Cobalt', 'Lead', 'Rubidium', 'Wollastonite', 'Copper', 'Lime', 'Salt', 'Yttrium', 'Diamond', 'Lithium', 'Sand and Gravel', 'Zeolites', 'Diatomite', 'Magnesium', 'Scandium', 'Zinc', 'Feldspar', 'Manganese', 'Selenium'])

In [17]:
scraping_base_bricolage = {k: v for k, v in list(scraping_base.items()) if k != 'Palladium' and k != 'Zirconium'}

In [18]:
output_pdf_path = 'pdf_output/mcs2024_data_all_tables.pdf'

draw_rectangles_for_materials(pdf_path, output_pdf_path, scraping_base_bricolage)

print(f"Rectangles drawn for materials in {pdf_path} and saved to {output_pdf_path}.")

Rectangles drawn for materials in mcs2024.pdf and saved to pdf_output/mcs2024_data_all_tables.pdf.


## Keep useful table in pdf and excel

In [19]:
def convert_dict_to_df(d):
    for key, value in d.items():
        if isinstance(value, dict):
            d[key] = convert_dict_to_df(value)
        elif isinstance(value, list):
            if all(isinstance(item, dict) for item in value):
                d[key] = pd.DataFrame(value)
            else:
                d[key] = [convert_dict_to_df(item) if isinstance(item, dict) else item for item in value]
    return d

In [20]:
def table_contains_target_phrases(table, target_phrases):
    return any(any(phrase.lower() in str(cell).lower() for cell in row) for row in table.values for phrase in target_phrases)

# Load the JSON file
with open('json_files/scraping_base_with_tables.json', 'r', encoding='utf-8') as json_file:
    loaded_scraping_base = json.load(json_file)

# Convert dictionary to DataFrame where applicable
scraping_base = convert_dict_to_df(loaded_scraping_base)

In [21]:
# Target phrases
target_phrases = ['World total (rounded)', 'World total']


# Filter the tables
for material in scraping_base.keys():
    new_dfs = []
    for table_data in scraping_base[material]['tables']:
        table, bbox, page = table_data
        df = pd.DataFrame(table)
        # Check if any cell in the dataframe matches the target phrases
        if df.apply(lambda row: any(match_element_in_text(target_phrases, str(cell)) for cell in row), axis=1).any():
            new_dfs.append(table_data)
            print(table_data)
            print("="*40)
            print()
        # if isinstance(table, pd.DataFrame) and table_contains_target_phrases(table, target_phrases):
        #     filtered_tables.append(table_data)
    
    # Update the 'tables' key with the filtered tables
    scraping_base[material]['tables'] = new_dfs     

# Save the filtered data as a new JSON file
with open('json_files/scraping_base_with_needed_tables.json', 'w', encoding='utf-8') as json_file:
    json.dump(scraping_base, json_file, indent=2, default=lambda x: x.to_dict() if isinstance(x, pd.DataFrame) else str(x))

print("Filtered JSON file created successfully.")

[[{'text': None, 'bottom': 323.68752, 'text_1': 'Fused aluminum oxide', 'text_2': 'Fused aluminum oxide', 'text_3': 'Silicon carbide', 'text_4': 'Silicon carbide'}, {'text': None, 'bottom': 334.66751999999997, 'text_1': '2022', 'text_2': '2023', 'text_3': '2022', 'text_4': '2023'}, {'text': 'United States', 'bottom': 345.1965, 'text_1': '—', 'text_2': '—', 'text_3': '40,000', 'text_4': '40,000'}, {'text': 'United States and Canada', 'bottom': 356.17650000000003, 'text_1': '60,000', 'text_2': '60,000', 'text_3': '—', 'text_4': '—'}, {'text': 'Australia', 'bottom': 367.1565, 'text_1': '50,000', 'text_2': '50,000', 'text_3': '—', 'text_4': '—'}, {'text': 'Austria', 'bottom': 378.1965, 'text_1': '90,000', 'text_2': '90,000', 'text_3': '—', 'text_4': '—'}, {'text': 'Brazil', 'bottom': 389.17650000000003, 'text_1': '50,000', 'text_2': '50,000', 'text_3': '40,000', 'text_4': '40,000'}, {'text': 'China', 'bottom': 400.1565, 'text_1': '800,000', 'text_2': '800,000', 'text_3': '450,000', 'text_4

In [22]:
pd.DataFrame(table_data[0])

,text,bottom,text_1,text_2,text_3
0,None,125.68752,Refinery productione,Refinery productione,Reserves
1,None,136.66752,2022,2023,None
2,United States,147.19650,W,W,"11,000"
3,Belgium,158.17650,200,200,—
4,Canada,169.15650,110,120,"6,000"
5,China,180.19650,"1,290","1,500","5,000"
6,Finland,191.17650,9130,100,300
7,Germany,202.15650,300,300,—
8,India,213.19650,14,14,500
9,Japan,224.17650,710,780,—


In [23]:
with open('json_files/scraping_base_with_needed_tables.json', 'r', encoding='utf-8') as json_file:
    loaded_scraping_base = json.load(json_file)

scraping_base = convert_dict_to_df(loaded_scraping_base)

In [24]:
for material in list(scraping_base.keys()):
    if 'remarks' in scraping_base[material]:
        remarks = scraping_base[material]['remarks']
        remarks_df = pd.DataFrame(remarks)
        scraping_base[material]['remarks'] = remarks_df

    if 'tables' in scraping_base[material]:    
        for i in range(len(scraping_base[material]['tables'])):
            table, bbox, page = scraping_base[material]['tables'][i]
            # Convert table to DataFrame
            df = pd.DataFrame(table)
            # Sort and drop 'bottom' column if it exists
            if 'bottom' in df.columns:
                df = df.sort_values('bottom')
                df = df.drop('bottom', axis=1)
            scraping_base[material]['tables'][i][0] = df

with pd.ExcelWriter('excel_output/production_reserve_tables1.xlsx', engine='openpyxl') as writer:
    for material, data in scraping_base.items():
        # print(f"🟠data: {data['remarks']}")
        # Write remarks and tables in the same sheet, but separated by columns
        combined_sheet_name = f"{material}_raw_info"
        
        # Write remarks
        remarks_df = data['remarks'][['text']]
        # print('🟠remark_df:', remarks_df)
        remarks_df.columns = ['remark']
        remarks_df.to_excel(writer, sheet_name=combined_sheet_name, startrow=0, startcol=0, index=False)

        # Write each table starting from a new column
        start_col = len(remarks_df.columns) + 2  # Adding 2 for separation
        for i, (table_df, bbox, page) in enumerate(data['tables']):
            table_df.to_excel(writer, sheet_name=combined_sheet_name, startrow=0, startcol=start_col, index=False)
            # Increment start_col by the number of columns in the table + 2 for separation
            start_col += len(table_df.columns) + 2

print("Excel file created successfully.")

Excel file created successfully.


In [25]:
with open('json_files/scraping_base_with_needed_tables.json', 'r', encoding='utf-8') as json_file:
# with open('json_files/scraping_base_with_tables.json', 'r', encoding='utf-8') as json_file:
    loaded_scraping_base = json.load(json_file)

scraping_base = convert_dict_to_df(loaded_scraping_base)

scraping_base_bricolage = {k: v for k, v in list(scraping_base.items()) if k != 'Palladium' and k != 'Zirconium'}

output_pdf_path = 'pdf_output/mcs2024_data_prod_reserve_tables.pdf'

draw_rectangles_for_materials(pdf_path, output_pdf_path, scraping_base_bricolage)

print(f"Rectangles drawn for materials in {pdf_path} and saved to {output_pdf_path}.")

Rectangles drawn for materials in mcs2024.pdf and saved to pdf_output/mcs2024_data_prod_reserve_tables.pdf.


In [26]:
STOP

NameError: name 'STOP' is not defined

## remark interpretation

In [ ]:
excel_input = 'excel_output\production_reserve_tables1.xlsx'

import json

with open('json_files/scraping_base.json', 'r', encoding='utf-8') as json_file:
    loaded_scraping_base = json.load(json_file)

scraping_base = convert_dict_to_df(loaded_scraping_base)

In [ ]:
material_remarks = {material: {'remarks': content['remarks'][['text']]} for material, content in scraping_base.items()}

In [ ]:
# # Convert the DataFrames in the dictionary
# serialize_dict(material_remarks)

# # Save the transformed dictionary as JSON
# with open('materials_data.json', 'w') as json_file:
#     json.dump(material_remarks, json_file, indent=4)

In [ ]:
ref_chem_compo = pd.read_excel('trash/sub_materials_database.xlsx', sheet_name='sub_materials_database')

metric_conversion_df = pd.read_excel('trash/sub_materials_database.xlsx', sheet_name='metric_conversion_factor')

In [ ]:
import pandas as pd
import shutil

# Copy the original Excel file to create an updated version
excel_output = 'excel_output/produc_reserve_remarkinterpre.xlsx'
shutil.copyfile(excel_input, excel_output)

# Open the Excel file for updating
with pd.ExcelWriter(excel_output, mode='a', if_sheet_exists='replace') as writer:
    for material in material_remarks.keys():
        # Convert the remarks to a DataFrame
        material_sample = pd.DataFrame(material_remarks[material]['remarks'], columns=['text'])

        # List of sub-material names from the reference_db
        materials_list = ref_chem_compo['sub_material_name'].tolist()

        # List of metric units from the metric_conversion_df
        metrics_list = metric_conversion_df['Metric Mentioned'].tolist()

        # Apply the functions to the 'text' column of material_sample
        material_sample['chemical_composition'] = material_sample['text'].apply(
            lambda text: find_all_chemical_compositions(text, materials_list, ref_chem_compo))
        material_sample['metric_conv_factor'] = material_sample['text'].apply(
            lambda text: find_metric_conversion_factor(text, metrics_list, metric_conversion_df))

        # Read the corresponding sheet from the original Excel file
        sheet_name = f"{material}_raw_info"
        original_df = pd.read_excel(excel_input, sheet_name=sheet_name)

        # Ensure the new DataFrame (material_sample) has the same index as the original_df
        material_sample = material_sample.reindex(original_df.index)

        # Insert the new columns next to the 'remark' column
        original_df.insert(1, 'chemical_composition', material_sample['chemical_composition'])
        original_df.insert(2, 'metric_conv_factor', material_sample['metric_conv_factor'])

        # Write the updated sheet back to the Excel file
        original_df.to_excel(writer, sheet_name=sheet_name, index=False)

## production of each material

In [ ]:
excel_input = 'excel_output/produc_reserve_remarkinterpre.xlsx'

In [ ]:
excel_file = pd.ExcelFile(excel_input)

# List to store dictionaries of sheet names and DataFrames
dfs = []

# Loop through all sheet names
for sheet_name in excel_file.sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    # Create a dictionary with sheet name and DataFrame
    sheet_dict = {
        f'sheet_name': sheet_name,
        f'df': df
    }
    
    # Append the dictionary to the list
    dfs.append(sheet_dict)

In [ ]:
reference_excel =  pd.ExcelFile('sub_materials.xlsx')

reference_df = pd.read_excel(reference_excel)

Structure the new df

In [ ]:
for i in range(1, len(dfs)):
    sheet_name = dfs[i]['sheet_name']
    df = dfs[i]['df']
    text_columns = [col for col in df.columns if col.startswith('text')]
    if text_columns:
        text_df = df[text_columns]
        first_row = text_df.iloc[0]
        second_row = text_df.iloc[1]
        title_row = list(text_df.columns)
        materials = list(pd.unique(first_row.dropna().values))
        list_chem_comp = []
        for material in materials:
            chem_comp = match_composition(material, reference_df)
            list_chem_comp.append({
                "material": material,
                "chem_comp": chem_comp,
            })

        if all(item['chem_comp'] is None for item in list_chem_comp):
            material = sheet_name.replace('_raw_info', '')
            list_chem_comp = [{
                "material": material,
                "chem_comp": match_composition(material, reference_df),
            }]

        if all(item['chem_comp'] is None for item in list_chem_comp):
            print("🧿 no match")
            print("materials : ", materials)
            print("list_chem_comp : ", list_chem_comp)
            print("=" * 50)
            print()
        else:
            list_chem_comp = convert_string_to_dict(list_chem_comp)
            df['chemical_composition'] = str(list_chem_comp)
            print(list_chem_comp)
            print("=" * 50)
            print()

            # Check if the material name is from the sheet name
            sheet_material = sheet_name.replace('_raw_info', '')
            is_sheet_material = any(item['material'] == sheet_material for item in list_chem_comp)

            if is_sheet_material:
                # Process all numeric columns for sheet-wide material
                for index, col in enumerate(title_row):
                    if text_df[col].iloc[2:].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna().any():
                        item = next(item for item in list_chem_comp if item['material'] == sheet_material)
                        years = second_row.iloc[index].split(',') if isinstance(second_row.iloc[index], str) else [second_row.iloc[index]]
                        process_column(df, text_df, title_row, index, item, years)

            else: 
                for index, col in enumerate(first_row):
                    for item in list_chem_comp:
                        if col == item['material']:
                            years = second_row.iloc[index].split(',') if isinstance(second_row.iloc[index], str) else [second_row.iloc[index]]
                            process_column(df, text_df, title_row, index, item, years)

    dfs[i]['df'] = df

In [ ]:
# Save all DataFrames to a single Excel file
output_file = 'excel_output/production_each_material.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for df_dict in dfs:
        sheet_name = df_dict['sheet_name']
        df = df_dict['df']
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"All processed DataFrames have been saved to {output_file}")